# Chicago: Data Cleaning 

In [1]:
# ! pip install langdetect

In [2]:
import pandas as pd
import ast
from langdetect import detect

pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)

In [3]:
listing = pd.read_csv('./data/Chicago_listings.csv')
review = pd.read_csv('./data/Chicago_reviews.csv')

In [4]:
print(f'Shape for Chicago Listings CSV: {listing.shape}')
print(f'Shape for Chicago Reviews CSV: {review.shape}')

Shape for Chicago Listings CSV: (6372, 74)
Shape for Chicago Reviews CSV: (267809, 6)


## Claning Review csv file

In [5]:
review.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2384,25218143,2015-01-09,14385014,Ivan,it's a wonderful trip experience. I didn't exc...
1,2384,28475392,2015-03-24,16241178,Namhaitou,This is my first trip using Airbnb. I was a li...
2,2384,30273263,2015-04-19,26101401,Patrick,The reservation was canceled 80 days before ar...
3,2384,30974202,2015-04-30,26247321,Cristina,Sólo puedo decir cosas buenas de Rebecca. La h...
4,2384,31363208,2015-05-04,31293837,SuJung,Rebecca was an absolutely wonderful host.\r\n\...


Dropping unnecessary columns

In [6]:
review.drop(columns = ['id',
                       'reviewer_id',
                       'reviewer_name'],
            inplace = True)

Removing all reviews that are not from 2019

In [7]:
review = review[(review['date'] >= '2019-01-01') & (review['date'] <= '2019-12-31')]

In [8]:
review.shape

(86016, 3)

Dropping all rows with null values

In [9]:
review.isnull().sum()

listing_id     0
date           0
comments      67
dtype: int64

In [10]:
review = review.dropna()

After doing some EDA these are some of the things I noticed needed to be addressed.

Remove all reviews that are less than 5 words.

In [11]:
review = review[review['comments'].str.count(' ') > 4]

Remove all reviews that are not in English

In [12]:
review.drop([i for i in review[review['comments'].apply(detect) != 'en'].index], inplace=True)

Remove '\n' since this is just an indicator for a line break.

In [13]:
review['comments'] = review['comments'].str.replace('\n', '')

Removing any numbers from the comments.

In [14]:
review['comments'] = review['comments'].replace('\d+', '', regex=True)

Removing reviews written in Asian languages.

In [15]:
review.drop([i for i in review[review['comments'].str.contains(r'[^\x00-\x7F]+') == True].index], inplace = True)

Removing rows where the comments were generated by AirBnb due to the host cancelling a booked reservation.

In [16]:
review.drop([i for i in review[review['comments'].str.contains('This is an automated posting') == True].index], inplace = True)

## Cleaning Listing csv file

In [17]:
listing.head()

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2384,https://www.airbnb.com/rooms/2384,20210217171619,2021-02-19,"Hyde Park - Walk to UChicago, 10 min to McCormick",Visiting The University of Chicago or attendin...,The apartment is less than one block from beau...,https://a0.muscache.com/pictures/acf6b3c0-47f2...,2613,https://www.airbnb.com/users/show/2613,Rebecca,2008-08-29,"Chicago, Illinois, United States",My 2 bdrm apartment is a 2nd floor walk-up in ...,within an hour,100%,100%,t,https://a0.muscache.com/im/pictures/user/8bb47...,https://a0.muscache.com/im/pictures/user/8bb47...,Hyde Park,1.0,1.0,"['email', 'phone', 'reviews', 'manual_offline'...",t,t,"Chicago, Illinois, United States",Hyde Park,NaN,41.78790,-87.58780,Private room in condominium,Private room,1,NaN,1 shared bath,1.0,1.0,"[""Free street parking"", ""Washer"", ""Fire exting...",$65.00,2,90,2,2,90,90,2.0,90.0,NaN,t,20,50,80,355,2021-02-19,181,3,0,2015-01-09,2020-10-29,100.0,10.0,10.0,10.0,10.0,10.0,10.0,R17000015609,f,1,0,1,0,2.43
1,4505,https://www.airbnb.com/rooms/4505,20210217171619,2021-02-19,394 Great Reviews. 127 y/o House. 40 yds to tr...,We have been closed for quite a while and will...,NaN,https://a0.muscache.com/pictures/10628083/3543...,5775,https://www.airbnb.com/users/show/5775,Craig & Kathleen,2008-12-29,"Chicago, Illinois, United States",Incredibly creative(Kathleen) and stunningly a...,within an hour,100%,88%,t,https://a0.muscache.com/im/users/5775/profile_...,https://a0.muscache.com/im/users/5775/profile_...,Little Village,1.0,1.0,"['email', 'phone', 'reviews', 'kba']",t,t,NaN,South Lawndale,NaN,41.85495,-87.69696,Entire apartment,Entire home/apt,7,NaN,1 bath,3.0,5.0,"[""Free street parking"", ""Garden or backyard"", ...",$117.00,2,60,2,2,60,60,2.0,60.0,NaN,t,0,0,21,184,2021-02-19,395,1,0,2009-03-06,2020-07-14,96.0,10.0,9.0,10.0,10.0,10.0,9.0,R19000037538,t,1,1,0,0,2.71
2,7126,https://www.airbnb.com/rooms/7126,20210217171619,2021-02-19,Tiny Studio Apartment 94 Walk Score,A very small studio in a wonderful neighborhoo...,"Ukrainian Village was just named ""Hottest Neig...",https://a0.muscache.com/pictures/51073/16c81c7...,17928,https://www.airbnb.com/users/show/17928,Sarah,2009-05-19,"Chicago, Illinois, United States",We live in Chicago. We love to travel and see ...,within a few hours,90%,93%,t,https://a0.muscache.com/im/users/17928/profile...,https://a0.muscache.com/im/users/17928/profile...,Ukrainian Village,2.0,2.0,"['email', 'phone', 'reviews', 'jumio', 'offlin...",t,t,"Chicago, Illinois, United States",West Town,NaN,41.90289,-87.68182,Entire apartment,Entire home/apt,2,NaN,1 bath,3.0,4.0,"[""Free street parking"", ""Smoke alarm"", ""Fire e...",$70.00,2,60,2,2,1125,1125,2.0,1125.0,NaN,t,30,60

In [18]:
listing.isnull().sum().sum()

42227

In [19]:
listing.isnull().sum()

id                                                 0
listing_url                                        0
scrape_id                                          0
last_scraped                                       0
name                                               0
description                                       18
neighborhood_overview                           1669
picture_url                                        0
host_id                                            0
host_url                                           0
host_name                                          3
host_since                                         3
host_location                                     11
host_about                                      2008
host_response_time                              1572
host_response_rate                              1572
host_acceptance_rate                             943
host_is_superhost                                  3
host_thumbnail_url                            

### Dropping Columns

Columns with no relevant information

In [20]:
listing.drop(columns = ['last_scraped',                         
                        'license',
                        'host_id',
                        'scrape_id',                            
                        'listing_url',                          
                        'picture_url',                          
                        'host_url',                             
                        'host_thumbnail_url',                   
                        'host_picture_url',                     
                        'host_name',                            
                        'host_verifications',
                        'calendar_last_scraped',
                        'host_neighbourhood',
                        'host_location',
                        'host_response_rate',
                        'availability_30',
                        'availability_60',
                        'availability_90',
                        'availability_365',
                        'number_of_reviews_ltm',
                        'number_of_reviews_l30d',
                        'calculated_host_listings_count',
                        'calculated_host_listings_count_entire_homes',
                        'calculated_host_listings_count_private_rooms',
                        'calculated_host_listings_count_shared_rooms'],
            inplace = True)

Almost all values are nulls

In [21]:
listing.drop(columns = ['calendar_updated',                     
                        'neighbourhood_group_cleansed',         
                        'bathrooms'],
            inplace = True)

Repeat values from other columns

In [22]:
listing.drop(columns = ['minimum_minimum_nights',               
                        'maximum_minimum_nights',               
                        'minimum_maximum_nights',              
                        'maximum_maximum_nights',               
                        'minimum_nights_avg_ntm',               
                        'maximum_nights_avg_ntm',               
                        'neighbourhood',
                        'host_total_listings_count',
                        'beds',
                        'room_type'],                       
            inplace = True)

Almost all values are the same

In [23]:
listing.drop(columns = ['host_has_profile_pic',
                        'has_availability'], 
            inplace = True)

Dropping column due to multicollinearity

In [24]:
listing.drop(columns = ['host_identity_verified',
                        'host_is_superhost'],
             inplace = True)

### Dropping Nulls

The rows with no description are also missing data for many other rows.

In [25]:
listing.dropna(subset=['description'], inplace = True)

Since these rows have no values for these columns, it is implied that they had no reviews.

In [26]:
listing.dropna(subset=['first_review',
                       'last_review',
                       'review_scores_rating',
                       'review_scores_accuracy',
                       'review_scores_cleanliness',
                       'review_scores_checkin',
                       'review_scores_communication',
                       'review_scores_location',
                       'review_scores_value',
                       'host_listings_count'],
               inplace = True)

### Imputing Nulls

Filling the nulls with 'No Content' because there are 1091 nulls for 'neighborhood_overview' and 1476 nulls for 'host_about'.

In [27]:
listing['neighborhood_overview'].fillna('No Content', inplace = True)
listing['host_about'].fillna('No Content', inplace = True)

Filling in the 1 null for 'bathroom_text' with '1 bath' since it is for a listing of a 1 bedroom apartment, it most likely has only 1 bathroom available.

In [28]:
listing['bathrooms_text'].fillna('1 bath', inplace = True)

Filling the null values for 'host_acceptance_rate' with the mean value. First, converting the values from percentages into floats.

In [29]:
listing['host_acceptance_rate'] = listing['host_acceptance_rate'].str.replace('%', '').astype('float')/100.0

In [30]:
avg_acceptance_rate = listing['host_acceptance_rate'].value_counts().mean()

In [31]:
listing['host_acceptance_rate'].fillna(avg_acceptance_rate, inplace = True)

Filling the null values for 'bedrooms' with '1.0' since that is the most frequent value.

In [32]:
listing['bedrooms'].fillna('1.0', inplace = True)

Filling the null values for 'host_response_time' with 'within a few hours' since that is the most reasonable.

In [33]:
listing['host_response_time'].isnull().sum()

1261

In [34]:
listing['host_response_time'].value_counts()

within an hour        2813
within a few hours     582
within a day           353
a few days or more     108
Name: host_response_time, dtype: int64

In [35]:
listing['host_response_time'].fillna('within a few hours', inplace = True)

## Save clean dataframes

Verify all nulls are dealt with.

In [36]:
listing.isnull().sum().sum()

0

In [37]:
review.isnull().sum().sum()

0

Checking the final amount of columns and rows for the final dataframe.

In [38]:
print(f'Shape for Chicago Listings CSV: {listing.shape}')
print(f'Shape for Chicago Reviews CSV: {review.shape}')

Shape for Chicago Listings CSV: (5117, 32)
Shape for Chicago Reviews CSV: (60204, 3)


Saving finalized dataframes as new CSV files.

In [39]:
listing.to_csv('./data/Chicago_Listings_Clean', index = False)
review.to_csv('./data/Chicago_Reviews_Clean', index = False)